In [1]:
import base64
import IPython
import json
import numpy as np
import os
import random
import requests
from io import BytesIO
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [2]:
# Load the dataset json
class CocoDataset():
    def __init__(self, cocos, image):
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                        'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                        'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                        'magenta', 'sienna', 'maroon']

        self.coco = cocos
        self.image = image

        #self.process_info()
        #self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        #print('Dataset Info:')
        #print('=============')
        #for key, item in self.info.items():
        #    print('  {}: {}'.format(key, item))

        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        #print('')

    def display_licenses(self):
        #print('Licenses:')
        #print('=========')

        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')

    def display_categories(self):
        #print('Categories:')
        #print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(cat_id, self.categories[cat_id]['name']))
            print('')

    def display_image(self, image, show_polys=True, show_bbox=True, show_labels=True, show_crowds=True, use_url=False):
        #print('Image:')
        #print('======')
        #if image_id == 'random':
        #    image_id = random.choice(list(self.images.keys()))
        image_id = self.coco[0]['image_id']
        iscrowd=0
        # Print the image info
        #for key, val in self.coco.items():
        #    print('  {}: {}'.format(key, val))

        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))

        else:
            image = PILImage.open(image)

        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = "data:image/png;base64, " + base64.b64encode(buffered.getvalue()).decode()

        # Calculate the size and adjusted display size
        max_width = 900
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height

        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        labels = {}
        #print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if iscrowd != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})

                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})

                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']['size']:
                    segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(str(segmentation_points).lstrip('[').rstrip(']'))
            #print(segm)
            polygons[i] = polygons_list

            if i < len(self.colors):
                poly_colors[i] = self.colors[i]
            else:
                poly_colors[i] = 'white'
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[i] = str(bbox_points).lstrip('[').rstrip(']')

            #labels[1] = (self.categories[self.coco[0]['category_id']], (bbox_points[0], bbox_points[1] - 4))

            # Print details
            #print('    {}:{}:{}'.format(1, poly_colors[1], self.categories[self.coco[0]['category_id']]))

        # Draw segmentation polygons on image
        html = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(img_str, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)

        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(points, fill_color, stroke_color)

        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)

        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)

        if show_labels:
            for seg_id, label in labels.items():
                color = poly_colors[seg_id]
                html += '<text x="{}" y="{}" style="fill:{}; font-size: 12pt;">{}</text>'.format(label[1][0], label[1][1], color, label[0])

        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html

    def process_info(self):
        self.info = self.coco['info']

    def process_licenses(self):
        self.licenses = self.coco['licenses']

    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        #for category in self.coco['categories']:
        cat_id = self.coco[0]['category_id']

        # Add category to the categories dict
        if cat_id not in self.categories:
            self.categories[cat_id] = self.coco[0]['category_id']
        else:
            print("ERROR: Skipping duplicate category id: {}".format(category))

        # Add category to super_categories dict
        #if super_category not in self.super_categories:
        #    self.super_categories[super_category] = {cat_id} # Create a new set with the category id
        #else:
        #    self.super_categories[super_category] |= {cat_id} # Add category id to the set

    def process_images(self):
        self.images = {}
        #for image in self.coco['images']:
        #image_id = image['id']
        #    if image_id in self.images:
        #        print("ERROR: Skipping duplicate image id: {}".format(image))
        #    else:
        self.images[self.coco[0]['image_id']] = self.image

    def process_segmentations(self):
        self.segmentations = {}
        image_id = self.coco[0]['image_id']
        for segmentation in self.coco:
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append({key : segmentation[key] for key in segmentation.keys() & {'segmentation', 'bbox'}})


In [3]:
#annotation_path = "/home/james/output/inference/coco_instances_results.json"
#image_dir = "/home/james/LIVECell/images/livecell_test_images/"
#json_relation = "/home/james/LIVECell/images/livecell_test_images/test_a.json"

annotation_path = "/home/james/LIVECell/images/data/empty/output/inference/coco_instances_results.json"
image_dir = "/home/james/LIVECell/images/unlabelled_data/empty/"
json_relation = "/home/james/LIVECell/images/data/empty/empty.json"

json_file = open(annotation_path)
coco = json.load(json_file)
json_file.close()
json_file = open(json_relation)
ref = json.load(json_file)
json_file.close()
i = 0
htmls=[]

while i<len(coco) :
    instances = []
    instance = coco[i]
    image_id = instance['image_id']
    for image in ref['images'] :
        if image['id'] == image_id :
            file_name = image['file_name']
    same_image = True
    while same_image :
        i+=1
        if i==len(coco) : break
        else :
            if coco[i]['image_id'] == image_id :
                instances.append(coco[i])
            else :
                same_image = False
    coco_dataset = CocoDataset(instances, file_name)
    html = coco_dataset.display_image(os.path.join(image_dir, file_name))
    htmls.append(html)

# coco_dataset.display_info()
# coco_dataset.display_licenses()
coco_dataset.display_categories()

In [4]:
IPython.display.HTML(htmls[0])

In [15]:
IPython.display.HTML(htmls[1])

In [16]:
IPython.display.HTML(htmls[1])